In [31]:
import pandas as pd
import random
from sqlalchemy import create_engine
import numpy as np

1. On charge harge les fichiers sources

In [32]:
# Charger les fichiers sources
file1 = 'data\\ObesityDataSet_raw_and_data_sinthetic.csv'
file2 = 'data\\lifestyle_sustainability_data.csv'
file3 = 'data\\DietSurvey_toJson.json'

# Charger les fichiers dans des DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_json(file3)

2. On code les fonctions dont on aura besoin par la suite

In [33]:
def assign_diet_type(fcvc):
    if fcvc >= 2.6:
        # Si FCVC >= 2.6, attribuer "Vegetarian" ou "Vegan" avec probabilité pour "Vegan" plus faible
        return np.random.choice(['Vegetarian', 'Vegan'], p=[0.8, 0.2])
    elif fcvc >= 2:
        # Si FCVC >= 2 et < 2.6, attribuer "Mostly Plant-Based"
        return 'Mostly Plant-Based'
    elif fcvc <= 1:
        # Si FCVC <= 1, attribuer "Mostly Animal-Based"
        return 'Mostly Animal-Based'
    else:
        # Sinon, attribuer "Balanced"
        return 'Balanced'

In [34]:
# Traitement pour le fichier 1 (ObesityDataSet_raw_and_data_sinthetic.csv)
df1['diet_type'] = df1['FCVC'].apply(assign_diet_type)

# Traitement pour le fichier 2 (lifestyle_sustainability_data.csv)
# Ici, diet_type est simplement la colonne 'DietType'
df2['diet_type'] = df2['DietType']

# Traitement pour le fichier 3 (DietSurvey_toJson.json)
# Extraire la réponse avant les parenthèses dans la question "What would best describe your diet:"
# Si la réponse contient une parenthèse, on prend la partie avant la parenthèse ; sinon, on garde la réponse telle quelle
df3['diet_type'] = df3['What would best describe your diet:'].apply(lambda x: x.split('(')[0].strip() if '(' in x else x.strip())


3. On regarde les différentes valeurs obtenues

In [35]:
# Afficher les différentes valeurs uniques possibles pour file2 et file3
file1_unique_values = df1['diet_type'].unique()
file2_unique_values = df3['diet_type'].unique()
file3_unique_values = df3['diet_type'].unique()

# Combiner toutes les valeurs uniques dans un seul ensemble pour éviter les doublons
unique_diet_values = set(file1_unique_values).union(file2_unique_values).union(file3_unique_values)

# Afficher les valeurs uniques combinées
print("Valeurs uniques combinées pour diet_type :", unique_diet_values)

Valeurs uniques combinées pour diet_type : {'Non-Vegetarian', 'Pollotarian', 'Balanced', 'Vegan', 'Mostly Animal-Based', 'Pollo-pescetarian', 'Mostly Plant-Based', 'Pescatarian', 'Vegetarian', 'Eggetarian'}


4. On fabrique le Dataframe correspondant

In [36]:
# Créer un DataFrame avec les valeurs uniques et un ID pour chaque régime
df_diet = pd.DataFrame(unique_diet_values, columns=['diet_type'])
df_diet['id_diet'] = df_diet.index + 1  # Assigner un id_diet unique à chaque régime

# Réorganiser les colonnes pour que 'id_diet' soit la première colonne
df_diet = df_diet[['id_diet', 'diet_type']]

# Afficher le DataFrame créé
print("\nDataFrame des régimes alimentaires :")
print(df_diet)



DataFrame des régimes alimentaires :
   id_diet            diet_type
0        1       Non-Vegetarian
1        2          Pollotarian
2        3             Balanced
3        4                Vegan
4        5  Mostly Animal-Based
5        6    Pollo-pescetarian
6        7   Mostly Plant-Based
7        8          Pescatarian
8        9           Vegetarian
9       10           Eggetarian


In [1]:
from db_utils import get_db_engine

# Obtenir le moteur de base de données
engine = get_db_engine()

In [39]:
df_diet.to_sql('Diet', engine, if_exists='append', index=False)

print("Les données ont été insérées avec succès dans la table 'Diet'.")


Les données ont été insérées avec succès dans la table 'Diet'.


C:\Users\user\AppData\Local\Temp\ipykernel_21168\2127336705.py:1: UserWarning: The provided table name 'Diet' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_diet.to_sql('Diet', engine, if_exists='append', index=False)
